# 📊 Evaluation Script

Let's start by reading the data

In [28]:
# DO NOT CHANGE
import cv2 as cv
import numpy as np
import glob
import time
from flask import Flask, request, jsonify
import pickle
from skimage import io
import matplotlib.pyplot as plt
import import_ipynb
from preprocessing import *

importing Jupyter notebook from preprocessing.ipynb
importing Jupyter notebook from utilities.ipynb


In [30]:
sift = cv.SIFT_create(
    nfeatures=200,  # Allow more keypoints
    contrastThreshold=0.005,  # Adjusted for better sensitivity
    nOctaveLayers=5,  # Adjusted for multi-scale detection
    edgeThreshold=10,  # More edge sensitivity
)

In [35]:
## 1. Fill x_test and y_test:
x_test = []
y_test = []

fonts = ["Scheherazade New", "Marhey", "Lemonada", "IBM Plex Sans Arabic"]

for font in fonts:
    for filename in sorted(glob.glob(f'/content/test/{font}/*.jpeg')):
        img = cv.imread(filename, cv.COLOR_BGR2RGB)
        x_test.append(img)
        y_test.append(fonts.index(font))

# 2. Convert them to Numpy arrays:
x_test = np.asarray(x_test)
y_test = np.asarray(y_test)

print("shape of x_data:", x_test.shape, "shape of y_data:", y_test.shape)
assert set(y_test) == {0, 1, 2, 3}

shape of x_data: (4, 496, 1230, 3) shape of y_data: (4,)


Here you define your predict function following this specific interface:

In [23]:
'''
This function takes an RGB image of dimensions (1181, 1181, 3) from the test set and returns integer prediction ∈ {0,1,2,3}
'''
def make_prediction(x):
    start_time = time.time()
    # DEFINE YOUR FUNCTION HERE AND DO NOT CHANGE ANYTHING ELSE IN THE NOTEBOOK
    k=1700
    svm_classifier = pickle.load(open("final_svm.pkl", "rb"))
    kmeans = pickle.load(open("final_kmeans.pkl", "rb"))
    corrected = Preprocessing(x)
    _, des = sift.detectAndCompute(corrected, None)
    histo = np.zeros(k)
    nkp = np.size(len(des))
    for d in des:
        idx = kmeans.predict([d])
        histo[idx] += 1/nkp # Because we need normalized histograms, I prefere to add 1/nkp directly
    histo=histo.reshape(1,-1)
    y_pred = svm_classifier.predict(histo)[0]
    return int(y_pred)

# img = io.imread(f"./test/M.jpg")  # Load the image
# label = make_prediction(img)
# print(label)
# # Fill your team number here
TEAM_NUM = "12"

Now let's compute the accuracy of the model:

In [37]:

y_pred = []
times=[]


for x in x_test:
    assert x.shape == (1181, 1181, 3)
    start_time = time.time()
    ŷ = make_prediction(x)
    end_time = time.time()
    y_pred.append(ŷ)
    times.append(end_time-start_time)

with open('results.txt', 'w') as r:
    for y in y_pred:
        r.write(str(y)+'\n')
with open('time.txt', 'w') as r:
    for t in times:
        r.write(" {:.3f}\n".format(t))